In [1]:
from math import floor
from dateutil.parser import parse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [62]:
import datetime

In [2]:
df = pd.read_csv("data/2019/Divvy_Trips_2019_full.csv.gz", compression="gzip")

C:\Users\79500\AppData\Local\Temp\ipykernel_3376\672813569.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/2019/Divvy_Trips_2019_full.csv.gz", compression="gzip")


In [3]:
df.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,21742443,2019-01-01 00:04:37,2019-01-01 00:11:07,2167,390.0,199,Wabash Ave & Grand Ave,84,Milwaukee Ave & Grand Ave,Subscriber,Male,1989.0
1,21742444,2019-01-01 00:08:13,2019-01-01 00:15:34,4386,441.0,44,State St & Randolph St,624,Dearborn St & Van Buren St (*),Subscriber,Female,1990.0
2,21742445,2019-01-01 00:13:23,2019-01-01 00:27:12,1524,829.0,15,Racine Ave & 18th St,644,Western Ave & Fillmore St (*),Subscriber,Female,1994.0
3,21742446,2019-01-01 00:13:45,2019-01-01 00:43:28,252,"1,783.0",123,California Ave & Milwaukee Ave,176,Clark St & Elm St,Subscriber,Male,1993.0
4,21742447,2019-01-01 00:14:52,2019-01-01 00:20:56,1170,364.0,173,Mies van der Rohe Way & Chicago Ave,35,Streeter Dr & Grand Ave,Subscriber,Male,1994.0


In [22]:
df["end_time"] = df["end_time"].apply(lambda x: parse(x))

In [23]:
df["start_time"] = df["start_time"].apply(lambda x: parse(x))

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194083 entries, 0 to 4194082
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   trip_id            int64         
 1   start_time         datetime64[ns]
 2   end_time           datetime64[ns]
 3   bikeid             int64         
 4   tripduration       object        
 5   from_station_id    int64         
 6   from_station_name  object        
 7   to_station_id      int64         
 8   to_station_name    object        
 9   usertype           object        
 10  gender             object        
 11  birthyear          float64       
dtypes: datetime64[ns](2), float64(1), int64(4), object(5)
memory usage: 384.0+ MB


In [41]:
df["end_time"].max()

Timestamp('2019-06-17 16:04:35')

# Потерянные велосипеды
Сгруппируем каждый датасет по bikeid и возьмем первую и последнюю поездку. После этого объеденим все датасеты в один и найдем самую последнюю поездку для каждого велосипеда. Вычтем из общего количества велосипедов велосипеды, последняя активность которых была в 2018-2019 => Узнаем кол-во потерянных/сломанных устройств

In [24]:
grouped_by_bike_2019 = (
    df.groupby("bikeid").agg({"start_time": "min", "end_time": "max"}).reset_index()
)

In [25]:
df_2 = pd.read_csv("data/2018/Divvy_Trips_2018_full.csv.gz", compression="gzip")
grouped_by_bike_2018 = df_2.groupby("01 - Rental Details Bike ID").agg(
    {
        "01 - Rental Details Local Start Time": "min",
        "01 - Rental Details Local End Time": "max",
    }
)
grouped_by_bike_2018 = grouped_by_bike_2018.reset_index()
grouped_by_bike_2018["01 - Rental Details Bike ID"] = grouped_by_bike_2018[
    "01 - Rental Details Bike ID"
].astype(int)
grouped_by_bike_2018 = grouped_by_bike_2018.rename(
    columns={
        "01 - Rental Details Bike ID": "bikeid",
        "01 - Rental Details Local Start Time": "start_time",
        "01 - Rental Details Local End Time": "end_time",
    }
)

C:\Users\79500\AppData\Local\Temp\ipykernel_3376\4142244348.py:1: DtypeWarning: Columns (1,2,4,6,8,9,10,13,14,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv("data/2018/Divvy_Trips_2018_full.csv.gz", compression="gzip")


In [26]:
df_2 = pd.read_csv("data/2017/Divvy_Trips_2017_full.csv.gz", compression="gzip")
grouped_by_bike_2017 = (
    df_2.groupby("bikeid").agg({"start_time": "min", "end_time": "max"}).reset_index()
)

In [27]:
df_2 = pd.read_csv("data/2016/Divvy_Trips_2016_full.csv.gz", compression="gzip")
grouped_by_bike_2016 = (
    df_2.groupby("bikeid")
    .agg({"starttime": "min", "stoptime": "max"})
    .rename(columns={"starttime": "start_time", "stoptime": "end_time"})
    .reset_index()
)

In [28]:
df_2 = pd.read_csv("data/2015/Divvy_Trips_2015_full.csv.gz", compression="gzip")
grouped_by_bike_2015 = (
    df_2.groupby("bikeid")
    .agg({"starttime": "min", "stoptime": "max"})
    .rename(columns={"starttime": "start_time", "stoptime": "end_time"})
    .reset_index()
)

In [29]:
df_2 = pd.read_csv("data/2014/Divvy_Trips_2014_full.csv.gz", compression="gzip")
grouped_by_bike_2014 = (
    df_2.groupby("bikeid")
    .agg({"starttime": "min", "stoptime": "max"})
    .rename(columns={"starttime": "start_time", "stoptime": "end_time"})
    .reset_index()
)

In [46]:
full_grouped = pd.concat(
    [
        grouped_by_bike_2014,
        grouped_by_bike_2015,
        grouped_by_bike_2016,
        grouped_by_bike_2017,
        grouped_by_bike_2018,
        grouped_by_bike_2019,
    ]
)

In [47]:
full_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31579 entries, 0 to 6258
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   bikeid      31579 non-null  int64 
 1   start_time  31579 non-null  object
 2   end_time    31579 non-null  object
dtypes: int64(1), object(2)
memory usage: 986.8+ KB


In [81]:
full_grouped["start_time"] = full_grouped["start_time"].astype('datetime64[D]')
full_grouped["end_time"] = full_grouped["end_time"].astype('datetime64[D]')

In [82]:
full_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31579 entries, 0 to 6258
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   bikeid      31579 non-null  int64         
 1   start_time  31579 non-null  datetime64[ns]
 2   end_time    31579 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 986.8 KB


In [104]:
# Получаем датасет с самой первой и последней поездкой для каждого велосипеда
full_grouped_result = full_grouped.groupby("bikeid").agg(
    {"start_time": "min", "end_time": "max"}
)

In [105]:
# Убираем выбросы - не может быть так, что первая поездка была позже последней
full_grouped_result = full_grouped_result[
    full_grouped_result["start_time"] < full_grouped_result["end_time"]
]

In [95]:
# full_grouped_result["fact_time_practice"] =  full_grouped_result["start_time"] + datetime.timedelta(days = 1460)
# max_data = datetime.datetime.strptime('2019-06-17', '%Y-%m-%d')
# full_grouped_result["fact_time_theory"] = full_grouped_result["fact_time_practice"].apply(lambda x: min(x, max_data))
# full_grouped_result["fact_time"] = full_grouped_result["end_time"] - full_grouped_result["start_time"]
# full_grouped_result = full_grouped_result.drop(columns='fact_time_practice') 

In [106]:
full_grouped_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6370 entries, 1 to 6471
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_time  6370 non-null   datetime64[ns]
 1   end_time    6370 non-null   datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 149.3 KB


In [128]:
# Сколько дней прошло с 1 до последней поездки велосипеда
full_grouped_result["fact_time"] = full_grouped_result["end_time"] - full_grouped_result["start_time"]

In [144]:
# Сколько дней  прошло после последней поездки велосипеда до предпологаемого скрока службы(парвая поездка + 3 года)
full_grouped_result["fact_time_practice"] =  full_grouped_result["start_time"] + datetime.timedelta(days = 1100) - full_grouped_result["end_time"]

In [145]:
result = full_grouped_result[full_grouped_result["fact_time_practice"] - full_grouped_result["fact_time"] < '90 days']

In [146]:
lost_bicycle = full_grouped_result.shape[0] - result.shape[0]
lost_bicycle # Кол-во потерянных или сломанных велосипедов в период с 2013 по 2018-2019

222